# Data Science Test Widget Brain.

This assignment focuses on transshipments of a certain port. Each day, many vessels arrive in this port and are served by some stevedore(s). Four cargo types have been identified (ore, coal, oil, and petroleum), and vessels often carry a mixture of cargo types. For each unique vessel arrival (i.e. each row in the data), we would like a prediction of how much it transships (total of load & discharge activities) per cargo type. Variables of interest therefore are: discharge1, load1, discharge2, load2, discharge3, load3, discharge4 and load4. 

The data for this case is stored in ‘VesselData.csv’ and contains historical data. 

We would like you to provide us with a Jupyter notebook or Python script with the results of your endeavors, well enriched with comments elaborating on the steps taken, even if they did not lead you anywhere, and try to motivate your actions as much as possible. We would like to understand the approach you have taken and your line of thought.

Explanation of variables in the data:

| Variable | Explanation   |
|:-----|:-----|
|   eta  | Estimated time of arrival of vessel|
|  ata  | Actual time of arrival of vessel |
|atd	 |           Actual time of departure of vessel |
|vesseldwt|	    Vessel deadweight tonnage|
|vesseltype|	    Vessel type|
|discharge[x]|	Discharge amount of cargo type x|
|load[x]	  |      Load amount of cargo type x|
|stevedorenames|	(Anonymized) stevedore ID’s visited by vessel|
|hasnohamis	   | Boolean whether vessel has the HaMIS notification system|
|earliesteta|	    Estimated time of arrival of first entry to port|
|latesteta	 |   Estimated time of arrival of last entry to port (vessel can spread transshipment(s) over multiple days)|
|traveltype	  |  Travel type|
|previousportid|	ID of previous port|
|nextportid	   | ID of next port|
|isremarkable|	Boolean whether there is anything remarkable regarding the vessel|
|vesselid	  |  Vessel ID|

In [ ]:
#BEFORE STARTING:
#goal: prediction (forecasting) of load and discharge
#idea: use different time series approaches? or predict by using different vessel types and attributes?
#---> choose for regression way (no time series)

In [18]:
#imports
import pandas as pd #for dataframes
import matplotlib.pyplot as plt #for plotting
import numpy as np

In [99]:
#Import and show data
data = pd.read_csv("VesselData.csv", sep = ",")
data.head()

,eta,ata,atd,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,...,load4,stevedorenames,hasnohamis,earliesteta,latesteta,traveltype,previousportid,nextportid,isremarkable,vesselid
0,2017-09-19 00:00:00+00,2017-09-19 00:00:00+00,2017-09-22 00:00:00+00,109290.0,5,0,0,0,0,90173,...,0,Stevedore_104,NaN,2017-09-19 00:00:00+00,2017-09-19 00:00:00+00,ARRIVAL,981,731,f,2242
1,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-03 00:00:00+00,67170.0,3,0,0,0,0,0,...,0,Stevedore_109,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,19,15,f,5462
2,2017-09-30 00:00:00+00,2017-09-30 00:00:00+00,2017-10-01 00:00:00+00,67737.0,3,0,0,0,0,0,...,0,Stevedore_57,NaN,2017-09-30 00:00:00+00,2017-09-30 00:00:00+00,ARRIVAL,19,19,f,5251
3,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-03 00:00:00+00,43600.0,3,0,0,0,0,0,...,0,Stevedore_57,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,15,18,f,5268
4,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,9231.0,3,0,0,0,0,0,...,0,Stevedore_98,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,74,27,f,5504


In [86]:
#First investigations
print("number of rows is", len(data)) #number of rows
data.dtypes #datatypes

number of rows is 8208


eta                object
ata                object
atd                object
vesseldwt         float64
vesseltype          int64
discharge1          int64
load1               int64
discharge2          int64
load2               int64
discharge3          int64
load3               int64
discharge4          int64
load4               int64
stevedorenames     object
hasnohamis        float64
earliesteta        object
latesteta          object
traveltype         object
previousportid      int64
nextportid          int64
isremarkable       object
vesselid            int64
dtype: object

In [91]:
data.describe()

#NOTES:
# 4 (petroleum) has by far largest mean load ---> pay attention to scaling later on 
# all equal counts
#note that very often amount is 0 (75% is 0 for all 4 categories)

,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,load3,discharge4,load4,hasnohamis,previousportid,nextportid,vesselid
count,8206.000000,8208.000000,8208.000000,8208.000000,8208.000000,8208.000000,8208.000000,8208.000000,8208.000000,8208.000000,0.0,8208.000000,8208.000000,8208.000000
mean,37929.072630,3.593811,1732.712841,60.754386,1168.698221,19.387549,4791.331871,44.745492,1820.782407,1509.046784,NaN,367.102461,379.989035,4336.194688
std,51742.798795,0.987787,16297.994648,1325.463945,11331.135301,665.333248,25366.035772,3071.717973,10034.936858,11936.519585,NaN,417.216981,421.691921,1692.321323
min,624.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2.000000,2.000000,2.000000
25%,6600.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,31.000000,25.000000,3238.000000
50%,13031.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,126.000000,127.000000,4684.000000
75%,46600.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,666.000000,766.000000,5399.250000
max,320805.000000,5.000000,204304.000000,41761.000000,189933.000000,43639.000000,299647.000000,271251.000000,183837.000000,293449.000000,NaN,1915.000000,1914.000000,7486.000000


In [158]:
#Investigate whether or not time series analysis is relevant

#plt.plot(data['ata'], data['load1'])
#Notice: doesn't seem like timeseries, try approach with attributes

In [5]:
#OLD: used for time series approach

##Create in and out of sample (to quantify forcasts). Use last 1208 as out of sample
#ins = data.iloc[0:7000] #in sample
#oos = data.iloc[7000:8208]

In [93]:
#Create in and out of sample (to quantify forecasts)

from sklearn.model_selection import train_test_split
[ins, oos] = train_test_split(data, shuffle=True)
ins.head()

,eta,ata,atd,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,...,load4,stevedorenames,hasnohamis,earliesteta,latesteta,traveltype,previousportid,nextportid,isremarkable,vesselid
181,2017-10-04 00:00:00+00,2017-10-04 00:00:00+00,2017-10-06 00:00:00+00,50806.0,2,0,0,0,0,0,...,0,Stevedore_61,NaN,2017-10-04 00:00:00+00,2017-10-04 00:00:00+00,ARRIVAL,475,981,f,7283
2538,2017-05-26 00:00:00+00,2017-05-26 00:00:00+00,2017-05-28 00:00:00+00,3539.0,3,0,0,0,0,0,...,0,Stevedore_26,NaN,2017-05-26 00:00:00+00,2017-05-26 00:00:00+00,ARRIVAL,34,92,f,1945
7971,2017-11-04 00:00:00+00,2017-11-04 00:00:00+00,2017-11-06 00:00:00+00,3300.0,3,0,0,0,0,0,...,0,"Stevedore_86,Stevedore_11",NaN,2017-11-04 00:00:00+00,2017-11-04 00:00:00+00,ARRIVAL,418,542,f,2170
766,2017-09-02 00:00:00+00,2017-09-02 00:00:00+00,2017-09-05 00:00:00+00,2950.0,3,0,0,0,0,0,...,0,Stevedore_72,NaN,2017-09-02 00:00:00+00,2017-09-02 00:00:00+00,ARRIVAL,88,16,f,335
6767,2017-09-15 00:00:00+00,2017-09-13 00:00:00+00,2017-09-18 00:00:00+00,113808.0,5,0,0,0,0,0,...,0,"Stevedore_59,Stevedore_114,Stevedore_35",NaN,2017-09-13 00:00:00+00,2017-09-16 00:00:00+00,SHIFT,127,1156,f,7212


In [95]:
#Clean data

#remove missing values from vesseldwt and load
inscleaned = ins[np.isfinite(ins['vesseldwt'])]
inscleaned = inscleaned[np.isfinite(inscleaned['load1'])]
inscleaned = inscleaned[np.isfinite(inscleaned['load2'])]
inscleaned = inscleaned[np.isfinite(inscleaned['load3'])]
inscleaned = inscleaned[np.isfinite(inscleaned['load4'])]

ooscleaned = oos[np.isfinite(oos['vesseldwt'])]
ooscleaned = ooscleaned[np.isfinite(ooscleaned['load1'])]
ooscleaned = ooscleaned[np.isfinite(ooscleaned['load2'])]
ooscleaned = ooscleaned[np.isfinite(ooscleaned['load3'])]
ooscleaned = ooscleaned[np.isfinite(ooscleaned['load4'])]

print("number of is rows is", len(inscleaned)) #number of rows is
print("number of os rows is", len(ooscleaned)) #number of rows oos


number of is rows is 6154
number of os rows is 2052


In [117]:
#First: keep it simple, so use LR
#version 1: 
#only first regress load on weigth, vessel type and intercept: 
#idea: try to make dummies for vesseltype and iclude in OLS

dummies = pd.get_dummies(inscleaned['vesseltype'])
pd.concat([inscleaned, dummies], axis = 1)
inscleaned.head()


,eta,ata,atd,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,...,load4,stevedorenames,hasnohamis,earliesteta,latesteta,traveltype,previousportid,nextportid,isremarkable,vesselid
181,2017-10-04 00:00:00+00,2017-10-04 00:00:00+00,2017-10-06 00:00:00+00,50806.0,2,0,0,0,0,0,...,0,Stevedore_61,NaN,2017-10-04 00:00:00+00,2017-10-04 00:00:00+00,ARRIVAL,475,981,f,7283
2538,2017-05-26 00:00:00+00,2017-05-26 00:00:00+00,2017-05-28 00:00:00+00,3539.0,3,0,0,0,0,0,...,0,Stevedore_26,NaN,2017-05-26 00:00:00+00,2017-05-26 00:00:00+00,ARRIVAL,34,92,f,1945
7971,2017-11-04 00:00:00+00,2017-11-04 00:00:00+00,2017-11-06 00:00:00+00,3300.0,3,0,0,0,0,0,...,0,"Stevedore_86,Stevedore_11",NaN,2017-11-04 00:00:00+00,2017-11-04 00:00:00+00,ARRIVAL,418,542,f,2170
766,2017-09-02 00:00:00+00,2017-09-02 00:00:00+00,2017-09-05 00:00:00+00,2950.0,3,0,0,0,0,0,...,0,Stevedore_72,NaN,2017-09-02 00:00:00+00,2017-09-02 00:00:00+00,ARRIVAL,88,16,f,335
6767,2017-09-15 00:00:00+00,2017-09-13 00:00:00+00,2017-09-18 00:00:00+00,113808.0,5,0,0,0,0,0,...,0,"Stevedore_59,Stevedore_114,Stevedore_35",NaN,2017-09-13 00:00:00+00,2017-09-16 00:00:00+00,SHIFT,127,1156,f,7212


In [118]:
#Estimate LM: quite poor forecaster
from sklearn.linear_model import LinearRegression
included = ['vesseldwt']

X_train = inscleaned[included]
X_test = ooscleaned[included]
y_train = inscleaned['load1']

lm = LinearRegression()
lmfit = lm.fit(X_train,y_train)
print('intercept: ',lm.intercept_)
print('coefficients: ',lm.coef_)


fcastLM = lm.predict(X_test)
#print(fcastLM)
#print(oos['load1'])
MAPE_LM = np.mean((np.abs(fcastLM - oos['load1'])/oos['load1']))
print('MAPE: ',MAPE_LM) 

intercept:  51.86997373192953
coefficients:  [0.00015452]
MAPE:  inf


In [119]:
#reason of poor estimation is (other than lack of regressors), that often the load of a specific category is 0, therefore, 
#first try to model which type of boats deliver which goods (and afterwards perform model of amount per type) 

In [139]:
ore = ins[(ins.load1 > 0) | (ins.discharge1 > 0)]
coal = ins[ins.load2 > 0 | (ins.discharge2 > 0)]
#print(coal)
oil = ins[ins.load3 > 0 | (ins.discharge3 > 0)]
petroleum = ins[ins.load4 > 0 | (ins.discharge4 > 0)]
print(len(ore), len(coal), len(oil), len(petroleum)) #number of rows

133 15 5 347


In [157]:
#SO: very specific group of oil (all of type 5, only 3 ships; 89,101,114) and coal transporters (all of type 3)
#focus on petroleum for now (largest group)
#Hypothesis: amount can be predicted by weight of both (natural upper bound on amount of petroleum) and previous and 
#next destinations (of course, using dummies would be better, but ill leave that out because of the time now).

petroleum_test = oos[oos.load4 > 0 | (oos.discharge4 > 0)]
print(len(petroleum_test))

#Q: Given a boat contains petroleum, waht amount do we predict?
included = ['vesseldwt', 'previousportid', 'nextportid']
X_train = petroleum[included]
y_train = petroleum['load4']

lm = LinearRegression()
lmfit = lm.fit(X_train,y_train)
print('intercept: ',lm.intercept_)
print('coefficients: ',lm.coef_)

X_test = petroleum_test[included]

fcastLM = lm.predict(X_test)
#print(fcastLM)
#print(oos['load1'])
MAPE_LM = np.mean((np.abs(fcastLM - petroleum_test['load4'])/petroleum_test['load4']))
print('MAPE: ',MAPE_LM) 



99
intercept:  -3355.471137332159
coefficients:  [ 0.8075254   0.52062362 -0.04925603]
MAPE:  0.44734869353557266


In [ ]:
#SO: first indication of amount is given. Remaining question: how to classify vessels to cargo type?
#easy approach: use a decision tree
from sklearn import tree
train_features = []
train_targets = []
tree = tree.DecisionTreeClassifier(criterion = 'entropy').fit(train_features,train_targets)

#idea: make 4 dummies for whether or not vessels transport the 4 cargo types (targets)
#use as features: vesseldwt, vesseltype, hasnohamis (and maybe more)